In [1]:
using HDF5, EzXML, Glob, Dates, Plots, Optim, LineSearches, LsqFit, JuMP, DSP, DataFrames, CSV, StatsBase, LinearRegression, LinearAlgebra, CurveFit, Symbolics
using HDBSCAN, Clustering
include("/Users/jjc/CSF/CSF Dynamics Scripts/readCSF.jl")

datapath = "/Users/jjc/Documents/SSC/recs_june_2022"
path = pwd();
savepath = "/Users/jjc/CSF/"
files = glob("*.hdf5", datapath);
# datapath = "/Users/jjc/CSF/Recordings"
# path = pwd();
# savepath = "/Users/jjc/CSF/"
# files = glob("*.hdf5", datapath);

for f = 1:length(files)
    filename = files[f]

    global Data = readCSF(filename)
    # sf = Data["samplingfr"]
    sf = 0.1
    println("File $f out of $(length(files))\n")

    global T = Data["T"]
    global rec_dur_s = Data["rec_dur_s"]
    global start_time = Data["start_time"]
    global end_time = Data["end_time"]
    global infusion_start_frame = Data["infusion_start_frame"]
    global infusion_end_frame = Data["infusion_end_frame"]
    global plateau_start = Data["plateau_start"]
    global plateau_end = Data["plateau_end"]
    global ICP = Data["ICP"]
    global E = Data["E"]
    global P_p = Data["P_p"]
    global P_b = Data["P_b"]
    global I_b = Data["I_b"]
    global I_inf = Data["I_inf"]
    global AMP = Data["AMP"]
    global P_0 = Data["P_0"]

    length(ICP) < infusion_end_frame ? Pm = ICP[infusion_start_frame:end] : Pm = ICP[infusion_start_frame:infusion_end_frame]

    # bayes_res = CSV.File("/Users/jjc/Documents/SSC/recs_june_2022/results_raw_jjc.csv")
    # bayes_res = CSV.File("/Users/jjc/Documents/SSC/recs_june_2022/GN_DIRECT_L.csv")
#   bayes_res = CSV.File("/Users/jjc/CSF.csv")
    icmp_res = CSV.File("/Users/jjc/Documents/SSC/recs_june_2022/results_raw_icm.csv")
    # icmp_res = CSV.File("/Users/jjc/CSF/Results/results_raw_icm.csv")
    bayes_res = CSV.File("/Users/jjc/CSF/Results/Results_BayesSTD.csv")
    # icmp_res = CSV.File("/Users/jjc/Documents/SSC/recs_june_2022/GN_DIRECT_L.csv")


    global R_icm = icmp_res["Rcsf"]
    global R_bayes = bayes_res["Rcsf"]

    global Ib_icm = icmp_res["Ib"]
    global Ib_bayes = bayes_res["Ib"]

    global P0_icm = icmp_res["P0"]
    global P0_bayes = bayes_res["P0"]

    global E_icm = icmp_res["E"]
    global E_bayes = bayes_res["E"]

    numsamples = length(ICP)
    infusion_end_frame > numsamples ? (global infusion_end_frame = numsamples) : 0
    P_m = zeros(numsamples)
    P_m .+= P_b

    ICPm = zeros(infusion_end_frame - infusion_start_frame)

    function getPm(I_b, E, P_0, ICP)
        err = 0
        P_model = zeros(numsamples)
        for i = infusion_start_frame:infusion_end_frame
            tᵢ = (i - infusion_start_frame) / 6
            It = I_b + I_inf
            ΔP = P_b - P_0
            y = It * ΔP / (I_b + (I_inf * exp(-E * It * tᵢ))) + P_0 + (Data["I_inf"] * Data["Rn"])
            P_model[i] = y
            err += (ICP[i] - P_model[i])^2
        end

        global goodness_of_fit = 100 * (sqrt(err) / length(P_model) / abs(mean(P_model)))

        ICPm = P_model[infusion_start_frame:infusion_end_frame]
        P_m[infusion_start_frame:infusion_end_frame] = ICPm
        P_m[infusion_end_frame+1:end] .= ICPm[end]
        return P_m, goodness_of_fit
    end


    colorscheme = "light"

    if colorscheme == "light"
        fgcolor = :black
        bgcolor = :white
        icpcol = :cadetblue
        bayescol = :red
        icmcol = :darkorchid4
        infcol = :gray95

    else
        fgcolor = :white
        bgcolor = RGB(0.13, 0.14, 0.14)
        icpcol = :cadetblue
        bayescol = :violet
        icmcol = :orange
        infcol = RGB(0.15, 0.17, 0.17)
    end

    nr = f
    framespan = plateau_end
    tix = [0:30:framespan;]
    h1 = vspan([infusion_start_frame, infusion_end_frame], color=infcol, legend=:outertopright, label="Infusion period", linecolor=:transparent)

    plot!(ICP, linecolor=icpcol, linewidth=3, label="ICP", alpha=0.7)
    Pm, gof_bayes = getPm(Ib_bayes[nr], E_bayes[nr], P0_bayes[nr], ICP)
    # plot!(Pm, linecolor=bayescol, linewidth=2, linestyle=:dash, xlims=[1, framespan],
    #     xlabel="Time [min]", ylabel="ICP [mmHg]", xticks=(tix, round.(tix ./ 6, digits=2)),
    #     label="Bayesian", grid=false, titlefontsize=8, titlealign=:left, background=bgcolor, foreground_color_subplot=fgcolor)

    Pm, gof_icm = getPm(Ib_icm[nr], E_icm[nr], P0_icm[nr], ICP)
    plot!(Pm, linecolor=icmcol, linewidth=2, linestyle=:dash, xlims=[1, framespan],
        xlabel="Time [min]", ylabel="ICP [mmHg]", xticks=(tix, round.(tix ./ 6, digits=2)),
        label="Gradient descent", grid=false, titlefontsize=8, titlealign=:left, background=bgcolor, foreground_color_subplot=fgcolor, ylims=[0.0, 50.0])

    txtcontent = ("Bayesian:" * "\n = $(round(Ib_bayes[nr], digits=2)) ± 0.15 [mL/min]\n" * "E = $(round(E_bayes[nr],digits=2)) ± 0.02 [mmHg/mL]\n" * "P₀ = $(round(P0_bayes[nr],digits=2)) ± 2.76 [mmHg]\n" * "Rₒ = $(round(R_bayes[nr],digits=2)) ± 3.4 [mmHg/mL/min]\n" * "err = $(round(gof_bayes, digits=3))\n" * "\nICM+:" * "\nIₐ = $(round(Ib_icm[nr], digits=2)) [mL/min]\n" * "E = $(round(E_icm[nr],digits=2)) [mmHg/mL]\n" * "P₀ = $(round(P0_icm[nr],digits=2)) [mmHg]\n" * "Rₒ = $(round(R_icm[nr],digits=2))  [mmHg/mL/min]\n" * "err = $(round(gof_icm, digits=3))\n")

    annotate!(framespan * 1.1, 10, text(txtcontent, fgcolor, :left, 7))

    global Pm = ICP[infusion_start_frame:infusion_end_frame]
    volRes, pressRes, fitted_curve, R2, MSE = press_vol_curve(R_bayes[nr], P0_bayes[nr])

    h2 = plot(volRes, log.(pressRes), seriestype=:scatter, makersize=5, color=bayescol, markerstrokewidth=0.5, markerstrokecolor=bgcolor, linewidth=1, label="R² = $(round(R2, digits=3))", grid=true, titlefontsize=8, titlealign=:left, background=bgcolor, size=(700, 700), xlabel="Volume [mL]", ylabel="(P - Pss)/(Pb - Pss) [1/1]")
    plot!(volRes, fitted_curve, lw=2, linecolor=:black, linestyle=:dash, legend=:outertopright, label="")

    volRes, pressRes, fitted_curve, R2, MSE = press_vol_curve(R_icm[nr], P0_icm[nr])
    plot!(volRes, log.(pressRes), seriestype=:scatter, makersize=5, color=icmcol, markerstrokewidth=0.5, markerstrokecolor=bgcolor, linewidth=1, label="R² = $(round(R2, digits=3))", grid=true, titlefontsize=8, titlealign=:left, background=bgcolor, size=(700, 700), xlabel="Volume [mL]", ylabel="(P - Pss)/(Pb - Pss) [1/1]")
    plot!(volRes, fitted_curve, lw=2, linecolor=:black, linestyle=:dash, legend=:outertopright, label="")

    plot(h1, h2, layout=(2, 1))

    savefig("/Users/jjc/Documents/SSC/recs_june_2022/combined/" * filename[length(datapath)+2:end-5] * "combined.pdf")
end

File 1 out of 96

File 2 out of 96

File 3 out of 96

File 4 out of 96

File 5 out of 96

File 6 out of 96

File 7 out of 96

File 8 out of 96

File 9 out of 96

File 10 out of 96

File 11 out of 96

File 12 out of 96

File 13 out of 96

File 14 out of 96

File 15 out of 96

File 16 out of 96

File 17 out of 96

File 18 out of 96

File 19 out of 96

File 20 out of 96

File 21 out of 96

File 22 out of 96

File 23 out of 96

File 24 out of 96

File 25 out of 96

File 26 out of 96

File 27 out of 96

File 28 out of 96

File 29 out of 96

File 30 out of 96

File 31 out of 96

File 32 out of 96

File 33 out of 96

File 34 out of 96

File 35 out of 96

File 36 out of 96

File 37 out of 96

File 38 out of 96

File 39 out of 96

File 40 out of 96

File 41 out of 96

File 42 out of 96

File 43 out of 96

File 44 out of 96

File 45 out of 96

File 46 out of 96

File 47 out of 96

File 48 out of 96

File 49 out of 96

File 50 out of 96

File 51 out of 96

File 52 out of 96

File 53 out of 96

Fi

In [1]:
using HypothesisTests, CSV, Plots, StatsBase, LaTeXStrings

optmethod= ["Results_BayesSTD", "Results_BayesPss", "Results_Bayes4params"]
for optm in optmethod
dfBayes = CSV.File("/Users/jjc/CSF/Results/"* optm * ".csv")
dfNLOpt = CSV.File("/Users/jjc/CSF/Results/results_raw_icm.csv")

params = ["error", "Rcsf", "Ib", "P0", "E", "R2"]
# param ="error"
for param in params
# tormv = Bool.(zeros(length(dfNLOpt[param])))

tormv = dfNLOpt[param].<24000
tormv = .~tormv
# tormv = tormv .|| isnan.(dfNLOpt[param])
# tormv = tormv .|| (dfNLOpt[param].<24000)
# tormv = tormv .|| (dfBayes[param].<=20)

# if param == "error"
    # tormv = Bool.(dfNLOpt[param].<=0.0)
# #     tormv = tormv .|| Bool.(dfBayes[param].<=0.0)
# #     tormv = tormv .|| Bool.(dfNLOpt[param].<=0.0)
#     # global tormv = tormv .|| (dfNLOpt[param] .>= 1000) 
#     # global tormv = tormv .|| isnan.(dfNLOpt[param])
# end
# tormv = Bool.(dfNLOpt[param] .>= 1000) 

println(param)
println("BayesOpt: ", round(mean(dfBayes[param][.~tormv]), digits=3), ", ", round(std(dfBayes[param][.~tormv]), digits=3))
println("ICM+: ", round(mean(dfNLOpt[param][.~tormv]), digits=3), ", ", round(std(dfNLOpt[param][.~tormv]), digits=3))
println("P-value: ", round(pvalue(OneSampleTTest(vec(dfBayes[param][.~tormv]), vec(dfNLOpt[param][.~tormv]))), digits=90))

# histogram(dfBayes[param][.~tormv],  bins=100, label="NLOpt")
# histogram!(dfNLOpt[param][.~tormv], bins=100, alpha=0.7, label="ICM+", legend=:outertopright, xlims=(0,2))


if param == "Rcsf"
    global xl = L"R_{out}"
    minv = minimum(dfBayes[param][.~tormv].-dfNLOpt[param][.~tormv])
    maxv = maximum(dfBayes[param][.~tormv].-dfNLOpt[param][.~tormv])
end
if param == "Ib"
    global xl = L"I_f"
    minv = minimum(dfBayes[param][.~tormv].-dfNLOpt[param][.~tormv])
    maxv = maximum(dfBayes[param][.~tormv].-dfNLOpt[param][.~tormv])
end
if param == "E"
    global xl = L"E"
    minv = minimum(dfBayes[param][.~tormv].-dfNLOpt[param][.~tormv])
    maxv = maximum(dfBayes[param][.~tormv].-dfNLOpt[param][.~tormv])
end
if param == "P0"
    global xl = L"R_{out}"
    minv = minimum(dfBayes[param][.~tormv].-dfNLOpt[param][.~tormv])
    maxv = maximum(dfBayes[param][.~tormv].-dfNLOpt[param][.~tormv])
end
if param == "error"
    global xl = "RMSE of the fit"
    minv = -5
    maxv = maximum(dfBayes[param][.~tormv].-dfNLOpt[param][.~tormv])
end
if param == "R2"
    global xl = "Coefficient of determination (R²) of P-V curve"
    minv = minimum(dfBayes[param][.~tormv].-dfNLOpt[param][.~tormv])
    maxv = maximum(dfBayes[param][.~tormv].-dfNLOpt[param][.~tormv])
end

npt = 100
rng = LinRange(minv, maxv, npt)
rng_tks = LinRange(minv, maxv, length(collect(0:10:100)))
rng_tks = round.(collect(rng_tks),digits=1)

h1 = fit(Histogram, dfBayes[param][.~tormv], rng)
h2 = fit(Histogram, dfNLOpt[param][.~tormv], rng)
H = fit(Histogram, dfBayes[param][.~tormv].-dfNLOpt[param][.~tormv], rng)

# plot(H, seriestype=:bar)
histogram(dfBayes[param][.~tormv].-dfNLOpt[param][.~tormv], xlabel=xl, xlim=(minv, maxv),legend=false)



title!("$(optm[9:end]) vs. ICM+")
savefig("/Users/jjc/CSF/Results/" * optm * "_" * param * ".pdf")
end
end
# h = h1.weights .- h2.weights

# plot(h ./ 1141, seriestype=:bar, xticks=(0:10:100, rng_tks), label="BayesOpt minus ICM+", xlabel=xl, ylabel="% values")

# end
# h11 = plot(h1, seriestype=:bar)
# h22 = plot(h2, seriestype=:bar)


# plot(h11, h22, layout=(2, 1), xlims=(0.0, 10.0), ylims=(0, 100))
# title!("$param")
# h
# h
# dfBayes[param][.~tormv].-dfNLOpt[param][.~tormv]

error
BayesOpt: 0.58, 0.348
ICM+: 1.423, 1.829
P-value: 2.2343011090558057e-54
Rcsf
BayesOpt: 17.251, 12.108
ICM+: 15.475, 10.356
P-value: 5.61268572653723e-13
Ib
BayesOpt: 0.433, 0.293
ICM+: 0.722, 0.885
P-value: 4.240006042040871e-30
P0
BayesOpt: 4.255, 7.871
ICM+: 2.757, 10.543
P-value: 5.143093907218292e-7
E
BayesOpt: 0.252, 0.2
ICM+: 0.183, 0.139
P-value: 7.78161137447319e-34
R2
BayesOpt: 0.813, 0.228
ICM+: 0.808, 0.225
P-value: 0.00947141651565306
error
BayesOpt: NaN, NaN
ICM+: 1.423, 1.829
P-value: NaN


InexactError: InexactError: trunc(Int64, NaN)

In [2]:
using HypothesisTests, CSV, Plots, StatsPlots, StatsBase, LaTeXStrings, DataFrames, OutlierDetection

# optmethod = ["Results_Bayes_untruncPV", "Results_BayesPss_untruncPV", "Results_BayesStaticP0_untruncPV"]
# optmethod = ["Results_NelderMead", "Results_NelderMead_StaticP0"]
optmethod= ["Results_BayesSTD", "Results_Bayes4params", "Results_BayesPss"]

method_labels = ["Bayes " * L"P_{ss} = P_0", "Bayes " * L"P_{ss} \neq P_0" * "(4 vars)", "Bayes with static " * L"P_0"]
params = ["error", "Rcsf", "Ib", "P0", "E", "R2"]

X = [[1], [3], [5]]
clrs = [:orange, :cyan4, :grey]

for param in params
    global i=1
    for optm in optmethod
        dfNLOpt = CSV.File("/Users/jjc/CSF/Results/results_raw_icm.csv")
        dfBayes = CSV.File("/Users/jjc/CSF/Results/" * optm * ".csv")

        tormv = dfNLOpt[param] .< 240000
        tormv = tormv .&& (.~isnan.(dfNLOpt[param]))
        tormv = tormv .&& (.~isnan.(dfBayes[param]))
        vectoplot = dfBayes[param][tormv] .- dfNLOpt[param][tormv]
        vecmad = mad(vectoplot) * 10

        if i == 1
            global h = violin(X[i], vectoplot, label=false, color=clrs[i], side=:left)
            global h = boxplot!(X[i] .+ 0.5, vectoplot, label=method_labels[i], color=clrs[i], bar_width=0.5)
        else
            global h = violin!(X[i], vectoplot, legend=:outertopright, label=false, xticks=(1:8, []), ylabel="BayesOpt minus ICM+", color=clrs[i], side=:left)
            global h = boxplot!(X[i] .+ 0.5, vectoplot, legend=:outertopright, label=method_labels[i], xticks=(1:8, []), ylabel="BayesOpt minus ICM+", color=clrs[i], titlefontsize=8, bar_width=0.5, xaxis=false, grid=false, markeredgecolor=:white, ylims=(-vecmad, vecmad))
        end
        global i+=1
    end
    plot!([0, 6], [0, 0], linestyle=:dash, label=L"y=0", color=:red)

    if param == "Rcsf"
        global xl = L"R_{out}"
    end
    if param == "Ib"
        global xl = L"I_f"
    end
    if param == "E"
        global xl = L"E"
    end
    if param == "P0"
        global xl = L"P_0"
    end
    if param == "error"
        global xl = "RMSE of the fit"
    end
    if param == "R2"
        global xl = "Coeff. of determination (R²) of P-V curve"
    end
    if param == "Pss"
        global xl = L"P_{ss}"
    end
    h
    title!(xl * " difference (full P-V curve used)")
    savefig("/Users/jjc/CSF/Results/" * param * ".pdf")
end

In [ ]:
using HypothesisTests, CSV, Plots, StatsPlots, StatsBase, LaTeXStrings, DataFrames, OutlierDetection

# optmethod = ["Results_Bayes_untruncPV", "Results_BayesPss_untruncPV", "Results_BayesStaticP0_untruncPV"]
# optmethod = ["Results_NelderMead", "Results_NelderMead_StaticP0"]
optmethod= ["Results_BayesSTD", "Results_Bayes4params", "Results_BayesPss"]

method_labels = ["Bayes " * L"P_{ss} = P_0", "Bayes " * L"P_{ss} \neq P_0" * "(4 vars)", "Bayes with static " * L"P_0"]
params = ["error", "Rcsf", "Ib", "P0", "E", "R2"]

X = [[1], [3], [5]]
clrs = [:orange, :cyan4, :grey]

for param in params
    global i=1
    for optm in optmethod
        dfNLOpt = CSV.File("/Users/jjc/CSF/Results/results_raw_icm.csv")
        dfBayes = CSV.File("/Users/jjc/CSF/Results/" * optm * ".csv")

        tormv = dfNLOpt[param] .< 240000
        tormv = tormv .&& (.~isnan.(dfNLOpt[param]))
        tormv = tormv .&& (.~isnan.(dfBayes[param]))
        vectoplot = dfBayes[param][tormv] .- dfNLOpt[param][tormv]
        vecmad = mad(vectoplot) * 10

        if i == 1
            global h = violin(X[i], vectoplot, label=false, color=clrs[i], side=:left)
            global h = boxplot!(X[i] .+ 0.5, vectoplot, label=method_labels[i], color=clrs[i], bar_width=0.5)
        else
            global h = violin!(X[i], vectoplot, legend=:outertopright, label=false, xticks=(1:8, []), ylabel="BayesOpt minus ICM+", color=clrs[i], side=:left)
            global h = boxplot!(X[i] .+ 0.5, vectoplot, legend=:outertopright, label=method_labels[i], xticks=(1:8, []), ylabel="BayesOpt minus ICM+", color=clrs[i], titlefontsize=8, bar_width=0.5, xaxis=false, grid=false, markeredgecolor=:white, ylims=(-vecmad, vecmad))
        end
        global i+=1
    end
    plot!([0, 6], [0, 0], linestyle=:dash, label=L"y=0", color=:red)

    if param == "Rcsf"
        global xl = L"R_{out}"
    end
    if param == "Ib"
        global xl = L"I_f"
    end
    if param == "E"
        global xl = L"E"
    end
    if param == "P0"
        global xl = L"P_0"
    end
    if param == "error"
        global xl = "RMSE of the fit"
    end
    if param == "R2"
        global xl = "Coeff. of determination (R²) of P-V curve"
    end
    if param == "Pss"
        global xl = L"P_{ss}"
    end
    h
    title!(xl * " difference (full P-V curve used)")
    savefig("/Users/jjc/CSF/Results/" * param * ".pdf")
end

In [ ]:
using HypothesisTests, CSV, Plots, StatsPlots, StatsBase, LaTeXStrings, DataFrames, OutlierDetection

# optmethod = ["Results_Bayes_untruncPV", "Results_BayesPss_untruncPV", "Results_BayesStaticP0_untruncPV"]
# optmethod = ["Results_NelderMead", "Results_NelderMead_StaticP0"]
optmethod= ["Results_BayesSTD", "Results_Bayes4params", "Results_BayesPss"]

method_labels = ["Bayes " * L"P_{ss} = P_0", "Bayes " * L"P_{ss} \neq P_0" * "(4 vars)", "Bayes with static " * L"P_0"]
params = ["error", "Rcsf", "Ib", "P0", "E", "R2"]

X = [[1], [3], [5]]
clrs = [:orange, :cyan4, :grey]

for param in params
    global i=1
    for optm in optmethod
        dfNLOpt = CSV.File("/Users/jjc/CSF/Results/results_raw_icm.csv")
        dfBayes = CSV.File("/Users/jjc/CSF/Results/" * optm * ".csv")

        tormv = dfNLOpt[param] .< 240000
        tormv = tormv .&& (.~isnan.(dfNLOpt[param]))
        tormv = tormv .&& (.~isnan.(dfBayes[param]))
        vectoplot = dfBayes[param][tormv] .- dfNLOpt[param][tormv]
        vecmad = mad(vectoplot) * 10

        if i == 1
            global h = violin(X[i], vectoplot, label=false, color=clrs[i], side=:left)
            global h = boxplot!(X[i] .+ 0.5, vectoplot, label=method_labels[i], color=clrs[i], bar_width=0.5)
        else
            global h = violin!(X[i], vectoplot, legend=:outertopright, label=false, xticks=(1:8, []), ylabel="BayesOpt minus ICM+", color=clrs[i], side=:left)
            global h = boxplot!(X[i] .+ 0.5, vectoplot, legend=:outertopright, label=method_labels[i], xticks=(1:8, []), ylabel="BayesOpt minus ICM+", color=clrs[i], titlefontsize=8, bar_width=0.5, xaxis=false, grid=false, markeredgecolor=:white, ylims=(-vecmad, vecmad))
        end
        global i+=1
    end
    plot!([0, 6], [0, 0], linestyle=:dash, label=L"y=0", color=:red)

    if param == "Rcsf"
        global xl = L"R_{out}"
    end
    if param == "Ib"
        global xl = L"I_f"
    end
    if param == "E"
        global xl = L"E"
    end
    if param == "P0"
        global xl = L"P_0"
    end
    if param == "error"
        global xl = "RMSE of the fit"
    end
    if param == "R2"
        global xl = "Coeff. of determination (R²) of P-V curve"
    end
    if param == "Pss"
        global xl = L"P_{ss}"
    end
    h
    title!(xl * " difference (full P-V curve used)")
    savefig("/Users/jjc/CSF/Results/" * param * ".pdf")
end

In [3]:
dfBayes = CSV.File("/Users/jjc/CSF/Results/Results_BayesPss.csv")
# dfBayes = CSV.File("/Users/jjc/CSF/Results/results_raw_icm.csv")



stdvar = dfBayes.Rcsf
stdvar = stdvar[.~isnan.(stdvar)]
println("Rcsf (mean) = $(round(mean(stdvar),digits=2))")
stdvar = dfBayes.Rcsf_std
stdvar = stdvar[.~isnan.(stdvar)]
println("Rcsf (std) = $(round(mean(stdvar),digits=2))")


stdvar = dfBayes.E
stdvar = stdvar[.~isnan.(stdvar)]
println("E (mean) = $(round(mean(stdvar),digits=2))")
stdvar = dfBayes.E_std
stdvar = stdvar[.~isnan.(stdvar)]
println("E (std) = $(round(mean(stdvar),digits=2))")


stdvar = dfBayes.Ib
stdvar = stdvar[.~isnan.(stdvar)]
println("I_b (mean) = $(round(mean(stdvar),digits=2))")
stdvar = dfBayes.Ib_std
stdvar = stdvar[.~isnan.(stdvar)]
println("I_b (std) = $(round(mean(stdvar),digits=2))")

stdvar = dfBayes.P0
stdvar = stdvar[.~isnan.(stdvar)]
println("P0 (mean) = $(round(mean(stdvar),digits=2))")
stdvar = dfBayes.P0_std
stdvar = stdvar[.~isnan.(stdvar)]
println("P0 (std) = $(round(mean(stdvar),digits=2))")


Rcsf (mean) = 16.86
Rcsf (std) = 0.88
E (mean) = 0.25
E (std) = 0.02
I_b (mean) = 0.49
I_b (std) = 0.03
P0 (mean) = 4.09
P0 (std) = 0.27


In [ ]:
using HDBSCAN, Clustering
    # dpress = zeros(length(P))
    # dvol = zeros(length(P))
    # # dpress[1] = 1
    # for i = 2:length(P)
    #     dvol[i] = dvol[i-1] + (I_inf + Ib_bayes[nr] - (P[i] - P0_bayes[nr]) / R_bayes[nr]) * 1 / 6
    #     dpress[i] = (P[i] - P0_bayes[nr]) / (P_b - P0_bayes[nr])
    # end

    # min_cluster_size=5
    # min_samples = 3

    # min_cluster_size = 100
    # min_samples = min_cluster_size

    # try
    # X = hcat(dvol, log.(dpress))'
    # result = hdbscan(X; min_cluster_size, min_samples)
    # global res = result.assignments
    # catch
    #     continue
    # end

    # h2 = plot(dvol[res.==0], (dpress[res.==0]), seriestype=:scatter, markersize=5, markerstrokewidth=0.5, markerstrokecolor=:black, linewidth=1, label="Model", grid=true, titlefontsize=8, titlealign=:left, background=bgcolor, size=(700, 800), xlabel="Volume [mL]", ylabel="(P - Pss)/(Pb - Pss) [1/1]", legend=false)
    # for i = 1:maximum(res)
    #     plot!(dvol[res.==i], (dpress[res.==i]), seriestype=:scatter, markerstrokewidth=0.5, markersize=5, markerstrokecolor=:black, linewidth=1, label="Model", grid=true, titlefontsize=8, titlealign=:left, background=bgcolor, xlabel="Volume [mL]", ylabel="(P - Pss)/(Pb - Pss) [1/1]", legend=:outertopright)
    # end

    # indCond = res .== 0
    # indCond = (res .== 1 .|| res .== 2)
    # volRes = dvol[indCond]
    # pressRes = dpress[indCond]

    # volRes = dvol[dpress.>0]
    # pressRes = dpress[dpress.>0]
    # coefval = CurveFit.curve_fit(LinearFit, volRes, log.(pressRes))
    # # fitvals = CurveFit.exp_fit(volRes, pressRes)
    # fitted_curve = coefval.(volRes)
    # y = log.(pressRes)
    # ydash = mean(y)
    # SSres = sum((y .- fitted_curve).^2)
    # SStot = sum((y .- ydash).^2)
    # R2 = 1 - (SSres/SStot)
    # MSE = SSres/length(y)
    # println(MSE)
X = hcat(dvol, log.(dpress))'

# min_cluster_size=5
# min_samples = 3

min_cluster_size=20
min_samples = min_cluster_size
result = hdbscan(X; min_cluster_size, min_samples)

probabilities(result)
exemplars(result)       # Computed at first call, this takes long time
outlier_scores(result)
raw_data(result) == X

res = result.assignments

plot(dvol[res .== 0], (dpress[res .== 0]), seriestype=:scatter, markerstrokewidth=.5, markerstrokecolor=:black, linewidth=1, label="Model", grid=true, titlefontsize=8, titlealign=:left, background=RGB(0.13, 0.14, 0.14), xlabel="Volume [mL]", ylabel="(P - Pss)/(Pb - Pss) [1/1]", legend=:outertopright)
for i = 1:maximum(res)
    plot!(dvol[res .== i], (dpress[res .== i]), seriestype=:scatter, markerstrokewidth=.5, markerstrokecolor=:black, linewidth=1, label="Model", grid=true, titlefontsize=8, titlealign=:left, background=RGB(0.13, 0.14, 0.14), xlabel="Volume [mL]", ylabel="(P - Pss)/(Pb - Pss) [1/1]", legend=:outertopright)
end

indCond = res.==0
# indCond = (res .== 1 .|| res .== 2)
volRes = dvol[indCond]
pressRes = dpress[indCond]
coefval = CurveFit.curve_fit(ExpFit, volRes, pressRes)
fitvals = CurveFit.exp_fit(volRes, pressRes)
fitted_curve = coefval.(volRes)

println(round(fitvals[2], digits=3))
plot!(volRes,fitted_curve, lw=3, linecolor=:orange, linestyle=:dash, yaxis=:log, legend=:outertopright)